In [1]:
from tm2py.acceptance.acceptance import Acceptance
from tm2py.acceptance.canonical import Canonical
from tm2py.acceptance.simulated import Simulated
from tm2py.acceptance.observed import Observed

In [2]:
scenario_config_file = "../examples/scenario_config.toml"
model_config_file = "../examples/model_config.toml"
observed_config_file = "../examples/observed_data.toml"
canonical_crosswalk_file = "../examples/canonical_crosswalk.toml"

output_folder_root = "/Users/wsp/Documents/GitHub/tm2py/examples/temp_acceptance/"

In [3]:
c = Canonical(canonical_crosswalk_file)

In [4]:
s = Simulated(canonical=c, scenario_file=scenario_config_file, model_file=model_config_file)

In [5]:
o = Observed(canonical=c, observed_file=observed_config_file)

In [6]:
a = Acceptance(canonical=c, simulated=s, observed=o, output_file_root=output_folder_root)

In [7]:
a.make_acceptance(make_transit=False, make_roadway=True, make_other=False)